In [1]:
### Developed by Renato Cezar, based on the script "getting_started.ipynb" supplied by THE FORAGE
### to perform an on the job training for BRITISH AIRWAYS
### Last modification: May 13th 2023

In [2]:
import requests # For web page scrapping
from bs4 import BeautifulSoup # For HTML parsing and handling
import pandas as pd # For data frames and data series
import re as re # For RegEx
import datetime # For date and time handling
from dateutil.parser import parse # For date and time parsing 
import nbimporter # To use outter python codes and notebooks
import matplotlib.pyplot as plt # To generate plots and graphs

In [3]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
# Set the number of pages to be scraped
pages = 20
# Set the number of registers per page
page_size = 100

parsed_content = []
lists_survey = []

# Loop to scrap content from each page:
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Pre Parse content
    content = response.content
    pre_parsed_content = BeautifulSoup(content, 'html.parser')
    
    # Split the articles from Pre Parsed Content and store them in a list of lists
    for article in pre_parsed_content.find_all("article", {"itemprop": "review"}):
        parsed_content.append(article)
    
    # Print the progress
    print(f"   ---> {len(parsed_content)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews
Scraping page 11
   ---> 1100 total reviews
Scraping page 12
   ---> 1200 total reviews
Scraping page 13
   ---> 1300 total reviews
Scraping page 14
   ---> 1400 total reviews
Scraping page 15
   ---> 1500 total reviews
Scraping page 16
   ---> 1600 total reviews
Scraping page 17
   ---> 1700 total reviews
Scraping page 18
   ---> 1800 total reviews
Scraping page 19
   ---> 1900 total reviews
Scraping page 20
   ---> 2000 total reviews


In [4]:
# Section to transform the data mass in parsed_content in a handsome dataset to be analyzed 
lists_survey = []
for i in range(0, len(parsed_content) - 1):
    # Gathering and organizing the data from the scrap
    # Date of survey
    date = parsed_content[i].find("time").contents[0]
    # Subject given by the surveyed
    subject = parsed_content[i].find(attrs={"class": "text_header"}).contents[0]
    # Name of the surveyed 
    name = parsed_content[i].find(attrs={"itemprop": "name"}).contents[0]
    # Country of the surveyed
    country = parsed_content[i].find("h3").contents[2]
    # Verification status of the surveyed trip 
    try:
        trip_verification = parsed_content[i].find("em").contents[0]
    except AttributeError:
        trip_verification = "NotAvailable" 
    # Commentary of the surveyed
    try:
        commentary = parsed_content[i].find(attrs={"class": "text_content"}).contents[2]
    except IndexError:
        try:
            commentary = parsed_content[i].find(attrs={"class": "text_content"}).contents[1]
        except IndexError:
            commentary = parsed_content[i].find(attrs={"class": "text_content"}).contents[0]
    # Aircraft model
    try:
        aircraft = parsed_content[i].find(attrs={"class": "review-rating-header aircraft"}).parent
        aircraft = aircraft.find(attrs={"class": "review-value"}).contents[0]
    except AttributeError:
        aircraft = "NotInformed"
    # Type of traveller
    try:
        type_traveller = parsed_content[i].find(attrs={"class": "review-rating-header type_of_traveller"}).parent
        type_traveller = type_traveller.find(attrs={"class": "review-value"}).contents[0]
    except AttributeError:
        seat_type = "NotInformed"
    # Seat/cabin type purchased
    try:
        seat_type = parsed_content[i].find(attrs={"class": "review-rating-header cabin_flown"}).parent
        seat_type =  seat_type.find(attrs={"class": "review-value"}).contents[0]
    except AttributeError:
        seat_type = "NotInformed"
    # Flight route
    try:
        route = parsed_content[i].find(attrs={"class": "review-rating-header route"}).parent
        route = route.find(attrs={"class": "review-value"}).contents[0]
        # Splitting the trip to catch separately "From", "To" and "Via"
        # First, verifity if theres is a "to"
        to = route.find(' to ') # if there is no "to", it will no be possible to securely splity this trip
        if to == -1: # So take all the string as "from"
            route_from = route
            route_to = "NotApplicable"
            route_via = "NotApplicable"
        else: # In case of existing a "to", check if there is a "via"
            via = route.find(' via ') # if there is no "via", just take the "from" and "to"
            if via == -1:
                route_from = route.split(" to ")[0]
                route_to = route.split(" to ")[1]
                route_via = "NotApplicable"
            else: # if there is a via, so take the "from", "to" and "via"
                route_from = route.split(" to ")[0]
                route_to_via = route.split(" to ")[1]
                route_to = route_to_via.split(" via ")[0]
                route_via = route.split(" via ")[1]
    except AttributeError:
        route_from = "NotInformed"
        route_to = "NotInformed"
        route_via = "NotInformed"
    # Date flown
    try:
        date_flown = parsed_content[i].find(attrs={"class": "review-rating-header date_flown"}).parent
        date_flown = date_flown.find(attrs={"class": "review-value"}).contents[0]
    except AttributeError:
        date_flown = "NotInformed"
    # Star ratings (coded to take only the maximum star rating)
    # Seat Comfort
    try:
        stars_seat_comfort = len((parsed_content[i].find(attrs={"class": "review-rating-header seat_comfort"}).parent).find_all("span", {"class": "star fill"}))
    except AttributeError:
        stars_seat_comfort = "NotEvaluated"
    # Cabin Staff Service
    try:      
        stars_cabin_staff_service = len((parsed_content[i].find(attrs={"class": "review-rating-header cabin_staff_service"}).parent).find_all("span", {"class": "star fill"}))
    except AttributeError:
        stars_cabin_staff_service = "NotEvaluated"    
    # Food and Beverage
    try:
        stars_food_beverage = len((parsed_content[i].find(attrs={"class": "review-rating-header food_and_beverages"}).parent).find_all("span", {"class": "star fill"}))
    except AttributeError:
        stars_food_beverage = "NotEvaluated"
    # Inflight Entertainment
    try:
        stars_inflight_entertainment = len((parsed_content[i].find(attrs={"class": "review-rating-header inflight_entertainment"}).parent).find_all("span", {"class": "star fill"}))
    except AttributeError:
        stars_inflight_entertainment = "NotEvaluated"
    # Ground Services
    try:
        stars_ground_services = len((parsed_content[i].find(attrs={"class": "review-rating-header ground_service"}).parent).find_all("span", {"class": "star fill"}))
    except AttributeError:
        stars_ground_services = "NotEvaluated"
    # WiFi and Connectivity
    try:
        stars_wifi_connectivity = len((parsed_content[i].find(attrs={"class": "review-rating-header wifi_and_connectivity"}).parent).find_all("span", {"class": "star fill"}))
    except AttributeError:
        stars_wifi_connectivity = "NotEvaluated"    
    # Value for Money
    try:
        stars_value_money = len((parsed_content[i].find(attrs={"class": "review-rating-header value_for_money"}).parent).find_all("span", {"class": "star fill"}))
    except AttributeError:
        stars_value_money = "NotEvaluated"
    # Recomended
    recommended = parsed_content[i].find(attrs={"class": "review-rating-header recommended"}).parent
    recommended = recommended.find(attrs={"class": re.compile("review-value rating-*")}).contents[0]

    # Storing the organized data as list in a set of lists
    lists_survey.append([date, subject, name, country, trip_verification, commentary, aircraft, type_traveller, 
                         seat_type, route_from, route_to, route_via, date_flown, stars_seat_comfort, 
                         stars_cabin_staff_service, stars_food_beverage, stars_inflight_entertainment, 
                         stars_ground_services, stars_wifi_connectivity, stars_value_money, recommended])

In [5]:
# Treating the text to remove any unnecessary chars (e.g: "|", "(" and ")") from Subject_Surveyer, Country_Surveyer and Commentary_Surveyer,
# formating the date in Survey_Date to the pattern "DD/MM/YYYY" and Date_Flown to "MM/YYYY"
for i in range(0, len(lists_survey)):
    #lists_survey[i][1] = lists_survey[i][1].replace('"', "")
    lists_survey[i][3] = lists_survey[i][3][lists_survey[i][3].find('(')+1 : lists_survey[i][3].find(')')]
    lists_survey[i][5] = lists_survey[i][5].replace("|", "")
    
    lists_survey[i][0] = parse(lists_survey[i][0]).strftime('%d/%m/%Y')
    lists_survey[i][12] = parse(lists_survey[i][12]).strftime('%m/%Y')

In [6]:
# Cleaning and uniformizing airport names (focusing in the London)
for i in range(0, len(lists_survey)):
    # Route_From
    lists_survey[i][9] = lists_survey[i][9].lstrip('"').rstrip('"')
    lists_survey[i][9] = lists_survey[i][9].lstrip(' ').rstrip(' ')
    lists_survey[i][9] = lists_survey[i][9].lstrip(',').rstrip(',')
    lists_survey[i][9] = lists_survey[i][9].replace("London Heathrow", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("London LHR", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("London LGW", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("London LCY", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("London City", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("London Cuty", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("London-Heathrow", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("London UK (Heathrow)", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("London Heaathrow", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("London Gatwick", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("Gatwick", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("Heathrow", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("LHR", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("LGW", "London")
    lists_survey[i][9] = lists_survey[i][9].replace("LCY", "London")
    # Route_To
    lists_survey[i][10] = lists_survey[i][10].lstrip('"').rstrip('"')
    lists_survey[i][10] = lists_survey[i][10].lstrip(' ').rstrip(' ')
    lists_survey[i][10] = lists_survey[i][10].lstrip(',').rstrip(',')
    lists_survey[i][10] = lists_survey[i][10].replace("London Heathrow", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("London LHR", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("London LGW", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("London LCY", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("London City", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("London Cuty", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("London-Heathrow", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("London UK (Heathrow)", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("London Heaathrow", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("London Gatwick", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("Gatwick", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("Heathrow", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("LHR", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("LGW", "London")
    lists_survey[i][10] = lists_survey[i][10].replace("LCY", "London")
    # Route_Via
    lists_survey[i][11] = lists_survey[i][11].lstrip('"').rstrip('"')
    lists_survey[i][11] = lists_survey[i][11].lstrip(' ').rstrip(' ')
    lists_survey[i][11] = lists_survey[i][11].lstrip(',').rstrip(',')
    lists_survey[i][11] = lists_survey[i][11].replace("London Heathrow", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("London LHR", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("London LGW", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("London LCY", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("London City", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("London Cuty", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("London-Heathrow", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("London UK (Heathrow)", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("London Heaathrow", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("London Gatwick", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("Gatwick", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("Heathrow", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("LHR", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("LGW", "London")
    lists_survey[i][11] = lists_survey[i][11].replace("LCY", "London")

In [7]:
# Creating and populating the data frame with the se of lists to handle the survey data in a tabular way
dataframe_survey = pd.DataFrame(lists_survey, columns=["Survey_Date", "Subject_Surveyer", "Name_Surveyer", "Country_Surveyer", 
                                 "Trip_Verification", "Commentary_Surveyer", "Aircraft", "Traveller_Type", 
                                 "Seat_Type", "Route_From", "Route_To", "Route_Via", "Date_Flown", "Seat_Comfort", 
                                 "Cabin_Services", "Food_Beverage", "Inflight_Entertainment", "Ground_Services", 
                                 "WiFi_Connectivity", "Value_Money","Recommended"])

In [8]:
# Enable the python helper script BA-Helper to use auxiliary functions
%run -i BA-Helper.py

In [9]:
# Call the function iata_converter, from BA-Helper, to get IATA, ICAO and airport names
dataframe_airport = iata_converter()
# Replace IATA codes in the client survey data frame to the airport name
dataframe_survey = dataframe_survey.replace((dataframe_airport[['IATA', 'Airport_Name']].set_index('IATA').T.to_dict('records'))[0])

C:\Users\renat\AppData\Local\Temp\ipykernel_9296\945477436.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dataframe_survey = dataframe_survey.replace((dataframe_airport[['IATA', 'Airport_Name']].set_index('IATA').T.to_dict('records'))[0])


In [10]:
# Save the data set as an Excel file
dataframe_survey.to_excel("BA_reviews-" + datetime.datetime.now().strftime("%d%b%Y_%H%M%S") + ".xlsx", sheet_name="reviews", index=True)

In [11]:
# Now that all data is nice and clean it's time to perform some basic statistics 

# Starting with getting the oldest and the earliest survey data date
# Transforming the date into an ordinal and building a list to take the higher and the 
# the smaller value, which are correspondant to the earliest and the oldest dates 
lists_dates = []
for i in range(0, len(dataframe_survey)):
    normal_date = dataframe_survey['Survey_Date'][i]
    ordinal_date = int(datetime.datetime.strptime(dataframe_survey['Survey_Date'][i], '%d/%m/%Y').toordinal())
    lists_dates.append([ordinal_date, normal_date])
    
dataframe_dates = pd.DataFrame(lists_dates, columns=["Ordinal_Date", "Normal_Date"])
list_earlier_oldest_date = [datetime.datetime.fromordinal(max(dataframe_dates["Ordinal_Date"])).strftime("%d/%m/%Y"), datetime.datetime.fromordinal(min(dataframe_dates["Ordinal_Date"])).strftime("%d/%m/%Y")]
serie_survey_dates = pd.Series(list_earlier_oldest_date, index = ["Earliest survey", "Oldest survey"], name = "Earlier_Oldest_Date")

In [12]:
# Now taking some data grouping and counting
group_count_country_surveyer = dataframe_survey.groupby("Country_Surveyer")["Country_Surveyer"].count().sort_values(ascending = False)
group_count_trip_verification = dataframe_survey.groupby("Trip_Verification")["Trip_Verification"].count().sort_values(ascending = False)
group_count_aircraft = dataframe_survey.groupby("Aircraft")["Aircraft"].count().sort_values(ascending = False)
group_count_traveller_type = dataframe_survey.groupby("Traveller_Type")["Traveller_Type"].count().sort_values(ascending = False)
group_count_seat_type = dataframe_survey.groupby("Seat_Type")["Seat_Type"].count().sort_values(ascending = False)
group_count_route_from = dataframe_survey.groupby("Route_From")["Route_From"].count().sort_values(ascending = False)
group_count_route_to = dataframe_survey.groupby("Route_To")["Route_To"].count().sort_values(ascending = False)
group_count_route_via = dataframe_survey.groupby("Route_Via")["Route_Via"].count().sort_values(ascending = False)
group_count_date_flown = dataframe_survey.groupby("Date_Flown")["Date_Flown"].count().sort_values(ascending = False)
group_count_recommended = dataframe_survey.groupby("Recommended")["Recommended"].count().sort_values(ascending = False)

In [14]:
print(serie_survey_dates)
print(group_count_country_surveyer)
print(group_count_trip_verification)
print(group_count_aircraft)
print(group_count_traveller_type)
print(group_count_seat_type)
print(group_count_route_from)
print(group_count_route_to)
print(group_count_route_via)
print(group_count_date_flown)
print(group_count_recommended)

Earliest survey    13/05/2023
Oldest survey      13/08/2016
Name: Earlier_Oldest_Date, dtype: object
Country_Surveyer
United Kingdom    1231
United States      240
Canada              64
Australia           63
Germany             42
                  ... 
Philippines          1
Panama               1
Israel               1
Jordan               1
Vietnam              1
Name: Country_Surveyer, Length: 64, dtype: int64
Trip_Verification
Trip Verified    1035
NotAvailable      771
Not Verified      193
Name: Trip_Verification, dtype: int64
Aircraft
NotInformed       739
A320              247
Boeing 777        169
A380              117
Boeing 747-400    100
                 ... 
A320-233            1
Boeing 744          1
787                 1
Boeing 737 800      1
Various             1
Name: Aircraft, Length: 144, dtype: int64
Traveller_Type
Couple Leisure    657
Solo Leisure      608
Business          462
Family Leisure    272
Name: Traveller_Type, dtype: int64
Seat_Type
Economy Class    